In [1]:
#importing all the libraries needed
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_selection import RFE
import statsmodels.api as sm
import random
import sklearn
from sklearn import metrics
import warnings

from sklearn import model_selection 
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

import matplotlib.pyplot as plt
import seaborn as sns #interactive plot
import os
import glob
from sklearn.utils import shuffle
from sklearn.model_selection import GridSearchCV
%matplotlib inline
warnings.filterwarnings(action='once')

In [2]:
path = r'C:\Users\Spelk\Desktop\Georgia Tech Analytics\Stephen Data Science\Data Portfolio\Machine Learning\NCAA Tourney Kaggle Competition\mens-machine-learning-competition-2019\DataFiles'
os.chdir(path)

In [3]:
#Download all files in wd
data = []

for counter,file in enumerate(os.listdir(path)):
    data.append(pd.read_csv(file))
print('All data loaded!')

All data loaded!


In [4]:
#Creating All dfs
Cities = data[0]
Conferences = data[1]
ConferenceTourneyGames = data[2]
GameCities = data[3]
NCAATourneyCompactResults = data[4]
NCAATourneyDetailedResults = data[5]
NCAATourneySeedRoundSlots = data[6]
NCAATourneySeeds = data[7]
NCAATourneySlots = data[8]
RegularSeasonCompactResults = data[9]
RegularSeasonDetailedResults = data[10]
Seasons = data[11]
SecondaryTourneyCompactResults = data[12]
SecondaryTourneyTeams = data[13]
TeamCoaches = data[14]
TeamConferences = data[15]
Teams = data[16]
Massey = data[17]
JP_df = data[18]

In [5]:
rankings = Massey.merge(Teams[['TeamID','TeamName']], how = 'left')
rankings.head(10)

,Season,RankingDayNum,SystemName,TeamID,OrdinalRank,TeamName
0,2003,35,SEL,1102,159,Air Force
1,2003,35,SEL,1103,229,Akron
2,2003,35,SEL,1104,12,Alabama
3,2003,35,SEL,1105,314,Alabama A&M
4,2003,35,SEL,1106,260,Alabama St
5,2003,35,SEL,1107,249,Albany NY
6,2003,35,SEL,1108,228,Alcorn St
7,2003,35,SEL,1110,204,American Univ
8,2003,35,SEL,1111,183,Appalachian St
9,2003,35,SEL,1112,26,Arizona


In [6]:
#Table that will be used to join the end of season rank for each team - by season and system
EOS_ranking_date = rankings.groupby(['TeamID','Season','SystemName'], as_index=False)['RankingDayNum'].max()

In [7]:
#Rankings for each team by season and system - these represent the ranking at the end of the regular season
EOS_ranking_by_system = EOS_ranking_date.merge(rankings, on = ['Season','TeamID','SystemName','RankingDayNum'],how = 'left')
#Drop Columns 'teamName'
EOS_ranking_by_system.drop(['TeamName','RankingDayNum'],axis=1,inplace=True)

Need to cast the rankings into individual columns for each year - the end dataset will be season, teamID, and a column for each ranking (100+)

In [8]:
EOS_ranking_by_system.head()

,TeamID,Season,SystemName,OrdinalRank
0,1101,2014,7OT,343
1,1101,2014,ADE,297
2,1101,2014,BBT,347
3,1101,2014,BIH,346
4,1101,2014,BLS,346


In [9]:
#Cast rankings
rankings_pivot = EOS_ranking_by_system.pivot_table(index = ['Season','TeamID'],columns = 'SystemName',values = 'OrdinalRank')
rankings_pivot.reset_index(drop = False,inplace=True)
rankings_pivot.head(5)

SystemName,Season,TeamID,7OT,ACU,ADE,AP,ARG,AUS,BBT,BCM,...,USA,WIL,WLK,WMR,WMV,WOB,WOL,WTE,YAG,ZAM
0,2003,1102,NaN,NaN,NaN,NaN,141.0,NaN,NaN,NaN,...,NaN,NaN,165.0,NaN,NaN,155.0,157.0,156.0,NaN,NaN
1,2003,1103,NaN,NaN,NaN,NaN,180.0,NaN,NaN,NaN,...,NaN,NaN,172.0,NaN,NaN,177.0,171.0,161.0,NaN,NaN
2,2003,1104,NaN,NaN,NaN,22.0,37.0,NaN,NaN,NaN,...,20.0,NaN,36.0,NaN,NaN,37.0,37.0,28.0,NaN,NaN
3,2003,1105,NaN,NaN,NaN,NaN,307.0,NaN,NaN,NaN,...,NaN,NaN,310.0,NaN,NaN,312.0,311.0,299.0,NaN,NaN
4,2003,1106,NaN,NaN,NaN,NaN,252.0,NaN,NaN,NaN,...,NaN,NaN,254.0,NaN,NaN,265.0,270.0,256.0,NaN,NaN


Only a few of the rankings have data for each year - I'm only going to use these for simplicity (COL - 6,DOL - 5,MOR - 1 ,POM - 4,RPI - 8,RTH - 1,WLK - 4,WOL - 1)

I'm also going to use AP and USA as indicator (binary) variables. 1 = in the AP rankings, and 0 = not in the rankings.


In [10]:
relevant_rankings = rankings_pivot[['Season','TeamID','COL','DOL','MOR','POM','RPI','RTH','WLK','WOL','AP','USA']]
relevant_rankings.head()

SystemName,Season,TeamID,COL,DOL,MOR,POM,RPI,RTH,WLK,WOL,AP,USA
0,2003,1102,162.0,175.0,132.0,160.0,158.0,146.0,165.0,157.0,NaN,NaN
1,2003,1103,172.0,174.0,139.0,163.0,182.0,168.0,172.0,171.0,NaN,NaN
2,2003,1104,43.0,39.0,26.0,33.0,38.0,31.0,36.0,37.0,22.0,20.0
3,2003,1105,310.0,315.0,309.0,307.0,313.0,312.0,310.0,311.0,NaN,NaN
4,2003,1106,256.0,266.0,294.0,263.0,248.0,274.0,254.0,270.0,NaN,NaN


In [11]:
#Creating binary variables for AP and USA - drop original variables inplace
relevant_rankings['AP_ind'] = np.where(relevant_rankings['AP'] > 0,1,0)
relevant_rankings['USA_ind'] = np.where(relevant_rankings['USA'] > 0,1,0)
relevant_rankings = relevant_rankings.drop(['AP','USA'],axis=1)

C:\Users\Spelk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [12]:
#Fill NAs with the mean of each column (if they're NA, they're most likely not in the tournament anyways)
relevant_rankings.fillna(relevant_rankings.mean(), inplace=True)

In [13]:
#Check for NAs
relevant_rankings.isna().sum()

SystemName
Season     0
TeamID     0
COL        0
DOL        0
MOR        0
POM        0
RPI        0
RTH        0
WLK        0
WOL        0
AP_ind     0
USA_ind    0
dtype: int64

In [14]:
NCAA_results = NCAATourneyCompactResults.drop(labels = ['WScore','LScore','WLoc','NumOT','DayNum'], axis = 1)
NCAA_results.head()

,Season,WTeamID,LTeamID
0,1985,1116,1234
1,1985,1120,1345
2,1985,1207,1250
3,1985,1229,1425
4,1985,1242,1325


In [15]:
#Set Min and Max Team ID - response variable = 1 if lower ID beats higher ID
NCAA_results['Low_TeamID'] = NCAA_results[['WTeamID','LTeamID']].min(axis=1)
NCAA_results['High_TeamID'] = NCAA_results[['WTeamID','LTeamID']].max(axis=1)
NCAA_results['Low_team_win'] = np.where(NCAA_results['Low_TeamID'] == NCAA_results['WTeamID'], 1, 0)
NCAA_results.drop(['WTeamID','LTeamID'],axis = 1, inplace=True)
NCAA_results.head()

,Season,Low_TeamID,High_TeamID,Low_team_win
0,1985,1116,1234,1
1,1985,1120,1345,1
2,1985,1207,1250,1
3,1985,1229,1425,1
4,1985,1242,1325,1


In [16]:
relevant_rankings.head()

SystemName,Season,TeamID,COL,DOL,MOR,POM,RPI,RTH,WLK,WOL,AP_ind,USA_ind
0,2003,1102,162.0,175.0,132.0,160.0,158.0,146.0,165.0,157.0,0,0
1,2003,1103,172.0,174.0,139.0,163.0,182.0,168.0,172.0,171.0,0,0
2,2003,1104,43.0,39.0,26.0,33.0,38.0,31.0,36.0,37.0,1,1
3,2003,1105,310.0,315.0,309.0,307.0,313.0,312.0,310.0,311.0,0,0
4,2003,1106,256.0,266.0,294.0,263.0,248.0,274.0,254.0,270.0,0,0


Convert the seeds to numbers with the function below

In [17]:
def seed_to_int(seed):
    #get the seed digits
    s_int = int(seed[1:3])
    return s_int
NCAATourneySeeds['seed_int'] = NCAATourneySeeds.Seed.apply(seed_to_int)
NCAATourneySeeds.drop(labels=['Seed'], inplace=True, axis=1) # This is the string label
NCAATourneySeeds.head()

,Season,TeamID,seed_int
0,1985,1207,1
1,1985,1210,2
2,1985,1228,3
3,1985,1260,4
4,1985,1374,5


Find seed difference for each matchup (Low_TeamID - High_TeamID)

In [18]:
NCAA_df = NCAA_results.merge(NCAATourneySeeds[['Season','TeamID','seed_int']], left_on=['Season','Low_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
NCAA_df2 = NCAA_df.merge(NCAATourneySeeds[['Season','TeamID','seed_int']], left_on=['Season','High_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
NCAA_df3 = NCAA_df2.drop(labels = ['TeamID_x','TeamID_y'],axis = 1)
NCAA_df3.rename(columns = {'seed_int_x':'Low_Seed','seed_int_y':'High_Seed'},inplace=True)
NCAA_df3['Seed_Dif'] = NCAA_df3['Low_Seed']-NCAA_df3['High_Seed']
NCAA_df3.drop(labels = ['Low_Seed','High_Seed'],axis = 1, inplace=True)

In [19]:
NCAA_df3.head()

,Season,Low_TeamID,High_TeamID,Low_team_win,Seed_Dif
0,1985,1116,1234,1,1
1,1985,1120,1345,1,5
2,1985,1207,1250,1,-15
3,1985,1229,1425,1,1
4,1985,1242,1325,1,-11


In [20]:
#Can only use tourney results from 2003-2018 --> don't have rankings back farther than that
mask_2003 = NCAA_df3['Season']>2002
NCAA_df4 = NCAA_df3[mask_2003]
NCAA_df4.head()

,Season,Low_TeamID,High_TeamID,Low_team_win,Seed_Dif
1136,2003,1411,1421,0,0
1137,2003,1112,1436,1,-15
1138,2003,1113,1272,1,3
1139,2003,1141,1166,1,5
1140,2003,1143,1301,1,-1


Perform the same opperations, but get the rank differences for each of the rankings (besides AP and USA - for those, I'll just use indicators.

In [21]:
def rank_difs(df,stats_df,system):
    #LowID team's rank
    df2 = df.merge(stats_df[['Season','TeamID',system]],left_on=['Season','Low_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
    #HighID team's rank
    df3 = df2.merge(stats_df[['Season','TeamID',system]],left_on=['Season','High_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
    #Drop multiple team_IDs
    df4 = df3.drop(labels = ['TeamID_x','TeamID_y'],axis = 1)
    
    #Calculate rank difference
    sys1 = system+'_x'
    sys2 = system+'_y'
    diff = system+'_diff'
    df4[diff] = df4[sys1]-df4[sys2]
    df4.drop(labels = [sys1,sys2],axis = 1, inplace=True)
    
    return df4

d1 = rank_difs(NCAA_df4,relevant_rankings,'COL')
d2 = rank_difs(d1,relevant_rankings,'DOL')
d3 = rank_difs(d2,relevant_rankings,'MOR')
d4 = rank_difs(d3,relevant_rankings,'POM')
d5 = rank_difs(d4,relevant_rankings,'RPI')
d6 = rank_difs(d5,relevant_rankings,'RTH')
d7 = rank_difs(d6,relevant_rankings,'WLK')
d8 = rank_difs(d7,relevant_rankings,'WOL')
d8.head()

,Season,Low_TeamID,High_TeamID,Low_team_win,Seed_Dif,COL_diff,DOL_diff,MOR_diff,POM_diff,RPI_diff,RTH_diff,WLK_diff,WOL_diff
0,2003,1411,1421,0,0,-1.0,-19.0,16.0,-5.0,31.0,6.0,3.0,26.0
1,2003,1112,1436,1,-15,-142.0,-154.0,-183.0,-141.0,-142.0,-180.0,-151.0,-175.0
2,2003,1113,1272,1,3,19.0,27.0,-1.0,12.0,6.0,16.0,13.0,20.0
3,2003,1141,1166,1,5,17.0,19.0,17.0,23.0,21.0,33.0,13.0,32.0
4,2003,1143,1301,1,-1,-28.0,-24.0,12.0,-8.0,-20.0,-12.0,-18.0,-21.0


Now bring in the indicators for each team (AP_ind and USA_ind)

In [22]:
#AP for both teams
d9 = d8.merge(relevant_rankings[['Season','TeamID','AP_ind']],left_on= ['Season','Low_TeamID'],
         right_on = ['Season','TeamID'], how = 'left')
d10 = d9.merge(relevant_rankings[['Season','TeamID','AP_ind']],left_on= ['Season','High_TeamID'],
         right_on = ['Season','TeamID'], how = 'left')

#USA for both teams
df11 = d10.merge(relevant_rankings[['Season','TeamID','USA_ind']],left_on= ['Season','Low_TeamID'],
         right_on = ['Season','TeamID'], how = 'left')
final_df = df11.merge(relevant_rankings[['Season','TeamID','USA_ind']],left_on= ['Season','High_TeamID'],
         right_on = ['Season','TeamID'], how = 'left')
final_df = final_df.drop(labels = ['TeamID_x','TeamID_y'],axis = 1)
final_df.head()

,Season,Low_TeamID,High_TeamID,Low_team_win,Seed_Dif,COL_diff,DOL_diff,MOR_diff,POM_diff,RPI_diff,RTH_diff,WLK_diff,WOL_diff,AP_ind_x,AP_ind_y,USA_ind_x,USA_ind_y
0,2003,1411,1421,0,0,-1.0,-19.0,16.0,-5.0,31.0,6.0,3.0,26.0,0,0,0,0
1,2003,1112,1436,1,-15,-142.0,-154.0,-183.0,-141.0,-142.0,-180.0,-151.0,-175.0,1,0,1,0
2,2003,1113,1272,1,3,19.0,27.0,-1.0,12.0,6.0,16.0,13.0,20.0,0,1,0,1
3,2003,1141,1166,1,5,17.0,19.0,17.0,23.0,21.0,33.0,13.0,32.0,0,1,0,1
4,2003,1143,1301,1,-1,-28.0,-24.0,12.0,-8.0,-20.0,-12.0,-18.0,-21.0,1,0,1,1


Need to convert "DayNum" to round number - this will be 0--> (0 is play in round, 6 is title game)

In [24]:
#Correlation matrix
final_df_preds = final_df.drop(['Season','Low_TeamID','High_TeamID'], axis =1)
final_df_preds.head()

,Low_team_win,Seed_Dif,COL_diff,DOL_diff,MOR_diff,POM_diff,RPI_diff,RTH_diff,WLK_diff,WOL_diff,AP_ind_x,AP_ind_y,USA_ind_x,USA_ind_y
0,0,0,-1.0,-19.0,16.0,-5.0,31.0,6.0,3.0,26.0,0,0,0,0
1,1,-15,-142.0,-154.0,-183.0,-141.0,-142.0,-180.0,-151.0,-175.0,1,0,1,0
2,1,3,19.0,27.0,-1.0,12.0,6.0,16.0,13.0,20.0,0,1,0,1
3,1,5,17.0,19.0,17.0,23.0,21.0,33.0,13.0,32.0,0,1,0,1
4,1,-1,-28.0,-24.0,12.0,-8.0,-20.0,-12.0,-18.0,-21.0,1,0,1,1


***Early submission file is 2014-2018 tourney results - use that as your test set***

In [25]:
#Split data into train and test sets
mask_2014 = final_df['Season']>2013
test = final_df_preds[mask_2014]
train = final_df_preds[-mask_2014]
print(train.shape)
print(test.shape)

(713, 14)
(335, 14)


In [26]:
'''Model used from one of the example kernals on kaggle'''
#logreg = LogisticRegression()
#params = {'C': np.logspace(start=-5, stop=3, num=9)}
#clf = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
#clf.fit(train, y_train)
#print('Best log_loss: {:.4}, with best C: {}'.format(clf.best_score_, clf.best_params_['C']))

'Model used from one of the example kernals on kaggle'

In [27]:
final_df_preds = final_df.drop(['Season','Low_TeamID','High_TeamID'], axis =1)
final_df_preds.head()
y_train = train.iloc[ :,0]
x_train = train.iloc[:,1:]
y_test = test.iloc[ :,0]
x_test = test.iloc[:,1:]

# Logistic Regression Model

In [28]:
logreg = LogisticRegression()
model = logreg.fit(x_train, y_train)
logit_model=sm.Logit(y_train,x_train)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.530910
         Iterations 7
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.234     
Dependent Variable: Low_team_win     AIC:              783.0783  
Date:               2019-03-18 11:06 BIC:              842.4815  
No. Observations:   713              Log-Likelihood:   -378.54   
Df Model:           12               LL-Null:          -494.01   
Df Residuals:       700              LLR p-value:      1.2687e-42
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     7.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Seed_Dif      -0.0988    0.0361  -2.7375  0.0062  -0.1695  -0.0281
COL_diff       0.0211    0.0143   1.4803  0.1388  -0.0068   0.

In [29]:
y_pred = logreg.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))

Accuracy of logistic regression classifier on test set: 0.71


In [30]:
#Log Loss Calc on Test Set
pred_prob = logreg.predict_proba(x_test)
print("Log Loss",sklearn.metrics.log_loss(y_test,pred_prob))

Log Loss 0.5657792646608648


In [33]:
train2 = train[['Low_team_win','Seed_Dif','COL_diff','MOR_diff','RPI_diff','RTH_diff','WLK_diff','WOL_diff',
               'AP_ind_x','AP_ind_y','USA_ind_x','USA_ind_y']]
test2 = test[['Low_team_win','Seed_Dif','COL_diff','MOR_diff','RPI_diff','RTH_diff','WLK_diff','WOL_diff',
               'AP_ind_x','AP_ind_y','USA_ind_x','USA_ind_y']]
y_train2 = train2.iloc[ :,0]
x_train2 = train2.iloc[:,1:]
y_test2 = test2.iloc[ :,0]
x_test2 = test2.iloc[:,1:]

In [34]:
logreg = LogisticRegression()
model = logreg.fit(x_train2, y_train2)
logit_model=sm.Logit(y_train2,x_train2)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.530976
         Iterations 7
                         Results: Logit
Model:              Logit            Pseudo R-squared: 0.234     
Dependent Variable: Low_team_win     AIC:              779.1717  
Date:               2019-03-18 11:07 BIC:              829.4360  
No. Observations:   713              Log-Likelihood:   -378.59   
Df Model:           10               LL-Null:          -494.01   
Df Residuals:       702              LLR p-value:      5.6959e-44
Converged:          1.0000           Scale:            1.0000    
No. Iterations:     7.0000                                       
------------------------------------------------------------------
               Coef.   Std.Err.     z     P>|z|    [0.025   0.975]
------------------------------------------------------------------
Seed_Dif      -0.0983    0.0361  -2.7235  0.0065  -0.1690  -0.0276
COL_diff       0.0220    0.0136   1.6204  0.1051  -0.0046   0.

In [35]:
y_pred = logreg.predict(x_test2)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test2, y_test2)))

Accuracy of logistic regression classifier on test set: 0.71


In [36]:
#Log Loss Calc on Test Set
pred_prob = logreg.predict_proba(x_test2)
print("Log Loss",sklearn.metrics.log_loss(y_test2,pred_prob))

Log Loss 0.5652704083820085


# Random Forest Model

In [37]:
clf=RandomForestClassifier(n_estimators=500,max_depth=10)

#Train the model using the training sets y_pred=clf.predict(X_test)
clf.fit(x_train,y_train)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=10, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)

In [38]:
rf_preds = clf.predict(x_test)
sum(rf_preds==y_test)/len(y_test)

0.7164179104477612

In [39]:
rf_probs = clf.predict_proba(x_test)
print("Log Loss:",sklearn.metrics.log_loss(y_test,rf_probs))

Log Loss: 0.5943869046794881


In [40]:
#Stats at the end of the regular season for every single team - includes last 10 stats
TeamID_max_game = JP_df.groupby(['Season','TeamID'],as_index=False)['Game'].max()
JP_df.columns.tolist()
JP2 = JP_df[['Season', 'TeamID', 'OppID', 'Game','Avg_PPP_Pre','Avg_PPP_L10','Avg_PPP_Allowed_Pre', 'Avg_PPP_Allowed_L10',
             'Avg_TS_Perc_Pre', 'Avg_TS_Perc_L10', 'Avg_TS_Perc_Allowed_Pre', 'Avg_TS_Perc_Allowed_L10',
             'Avg_Reb_Marg_Pre', 'Avg_Reb_Marg_L10', 'Avg_TO_Perc_Pre', 'Avg_Forced_TO_Perc_L10', 'Avg_Forced_TO_Perc_Pre']]
Max_game_stats = TeamID_max_game.merge(JP2,left_on=['Season','TeamID','Game'],right_on=['Season','TeamID','Game'], how = 'left')
Max_game_stats.columns.tolist()

['Season',
 'TeamID',
 'Game',
 'OppID',
 'Avg_PPP_Pre',
 'Avg_PPP_L10',
 'Avg_PPP_Allowed_Pre',
 'Avg_PPP_Allowed_L10',
 'Avg_TS_Perc_Pre',
 'Avg_TS_Perc_L10',
 'Avg_TS_Perc_Allowed_Pre',
 'Avg_TS_Perc_Allowed_L10',
 'Avg_Reb_Marg_Pre',
 'Avg_Reb_Marg_L10',
 'Avg_TO_Perc_Pre',
 'Avg_Forced_TO_Perc_L10',
 'Avg_Forced_TO_Perc_Pre']

In [41]:
#Merge with NCAA Tourney games - need to use final_df and the rank_diff function created above (system will be stat now)
def rank_difs(df,stats_df,system):
    #LowID team's rank
    df2 = df.merge(stats_df[['Season','TeamID',system]],left_on=['Season','Low_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
    #HighID team's rank
    df3 = df2.merge(stats_df[['Season','TeamID',system]],left_on=['Season','High_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
    #Drop multiple team_IDs
    df4 = df3.drop(labels = ['TeamID_x','TeamID_y'],axis = 1)
    
    #Calculate rank difference
    sys1 = system+'_x'
    sys2 = system+'_y'
    diff = system+'_diff'
    df4[diff] = df4[sys1]-df4[sys2]
    df4.drop(labels = [sys1,sys2],axis = 1, inplace=True)
    
    return df4

d1 = rank_difs(final_df,Max_game_stats,'Avg_PPP_Pre')
d2 = rank_difs(d1,Max_game_stats,'Avg_PPP_L10')
d3 = rank_difs(d2,Max_game_stats,'Avg_PPP_Allowed_Pre')
d4 = rank_difs(d3,Max_game_stats,'Avg_PPP_Allowed_L10')
d5 = rank_difs(d4,Max_game_stats,'Avg_TS_Perc_Pre')
d6 = rank_difs(d5,Max_game_stats,'Avg_TS_Perc_L10')
d7 = rank_difs(d6,Max_game_stats,'Avg_TS_Perc_Allowed_Pre')
d8 = rank_difs(d7,Max_game_stats,'Avg_TS_Perc_Allowed_L10')
d9 = rank_difs(d8,Max_game_stats,'Avg_Reb_Marg_Pre')
d10 = rank_difs(d9,Max_game_stats,'Avg_Reb_Marg_L10')
d11 = rank_difs(d10,Max_game_stats,'Avg_TO_Perc_Pre')
d12 = rank_difs(d11,Max_game_stats,'Avg_Forced_TO_Perc_Pre')
d13 = rank_difs(d12,Max_game_stats,'Avg_Forced_TO_Perc_L10')
d13.head()

,Season,Low_TeamID,High_TeamID,Low_team_win,Seed_Dif,COL_diff,DOL_diff,MOR_diff,POM_diff,RPI_diff,...,Avg_PPP_Allowed_L10_diff,Avg_TS_Perc_Pre_diff,Avg_TS_Perc_L10_diff,Avg_TS_Perc_Allowed_Pre_diff,Avg_TS_Perc_Allowed_L10_diff,Avg_Reb_Marg_Pre_diff,Avg_Reb_Marg_L10_diff,Avg_TO_Perc_Pre_diff,Avg_Forced_TO_Perc_Pre_diff,Avg_Forced_TO_Perc_L10_diff
0,2003,1411,1421,0,0,-1.0,-19.0,16.0,-5.0,31.0,...,-0.079583,-0.003585,-0.049993,-0.031205,0.013326,4.597291,7.0,-1.269225,2.338849,1.765797
1,2003,1112,1436,1,-15,-142.0,-154.0,-183.0,-141.0,-142.0,...,-0.004224,0.030790,0.043019,-0.028937,-0.027638,-0.513228,-5.3,-1.758093,1.554615,0.142975
2,2003,1113,1272,1,3,19.0,27.0,-1.0,12.0,6.0,...,0.136335,0.022790,0.016552,0.039895,0.097333,0.785714,0.9,0.438123,0.824697,1.527463
3,2003,1141,1166,1,5,17.0,19.0,17.0,23.0,21.0,...,0.043207,0.022643,0.095679,0.030167,0.017834,1.129464,4.7,5.416020,-1.840618,-0.317027
4,2003,1143,1301,1,-1,-28.0,-24.0,12.0,-8.0,-20.0,...,-0.043713,-0.023796,-0.032261,-0.013274,-0.003079,1.740148,0.3,-1.502949,-0.495638,3.062255


In [43]:
final_df_preds = d13.drop(['Season','Low_TeamID','High_TeamID'], axis =1)
final_df_preds.head()

,Low_team_win,Seed_Dif,COL_diff,DOL_diff,MOR_diff,POM_diff,RPI_diff,RTH_diff,WLK_diff,WOL_diff,...,Avg_PPP_Allowed_L10_diff,Avg_TS_Perc_Pre_diff,Avg_TS_Perc_L10_diff,Avg_TS_Perc_Allowed_Pre_diff,Avg_TS_Perc_Allowed_L10_diff,Avg_Reb_Marg_Pre_diff,Avg_Reb_Marg_L10_diff,Avg_TO_Perc_Pre_diff,Avg_Forced_TO_Perc_Pre_diff,Avg_Forced_TO_Perc_L10_diff
0,0,0,-1.0,-19.0,16.0,-5.0,31.0,6.0,3.0,26.0,...,-0.079583,-0.003585,-0.049993,-0.031205,0.013326,4.597291,7.0,-1.269225,2.338849,1.765797
1,1,-15,-142.0,-154.0,-183.0,-141.0,-142.0,-180.0,-151.0,-175.0,...,-0.004224,0.030790,0.043019,-0.028937,-0.027638,-0.513228,-5.3,-1.758093,1.554615,0.142975
2,1,3,19.0,27.0,-1.0,12.0,6.0,16.0,13.0,20.0,...,0.136335,0.022790,0.016552,0.039895,0.097333,0.785714,0.9,0.438123,0.824697,1.527463
3,1,5,17.0,19.0,17.0,23.0,21.0,33.0,13.0,32.0,...,0.043207,0.022643,0.095679,0.030167,0.017834,1.129464,4.7,5.416020,-1.840618,-0.317027
4,1,-1,-28.0,-24.0,12.0,-8.0,-20.0,-12.0,-18.0,-21.0,...,-0.043713,-0.023796,-0.032261,-0.013274,-0.003079,1.740148,0.3,-1.502949,-0.495638,3.062255


## Modeling Round 2

In [44]:
#Split data into train and test sets
mask_2014 = d13['Season']>2013
test = final_df_preds[mask_2014]
train = final_df_preds[-mask_2014]
print(train.shape)
print(test.shape)

(713, 27)
(335, 27)


In [82]:
train.columns.tolist

<bound method IndexOpsMixin.tolist of Index(['Low_team_win', 'Seed_Dif', 'COL_diff', 'DOL_diff', 'MOR_diff',
       'POM_diff', 'RPI_diff', 'RTH_diff', 'WLK_diff', 'WOL_diff', 'AP_ind_x',
       'AP_ind_y', 'USA_ind_x', 'USA_ind_y', 'Avg_PPP_Pre_diff',
       'Avg_PPP_L10_diff', 'Avg_PPP_Allowed_Pre_diff',
       'Avg_PPP_Allowed_L10_diff', 'Avg_TS_Perc_Pre_diff',
       'Avg_TS_Perc_L10_diff', 'Avg_TS_Perc_Allowed_Pre_diff',
       'Avg_TS_Perc_Allowed_L10_diff', 'Avg_Reb_Marg_Pre_diff',
       'Avg_Reb_Marg_L10_diff', 'Avg_TO_Perc_Pre_diff',
       'Avg_Forced_TO_Perc_Pre_diff', 'Avg_Forced_TO_Perc_L10_diff'],
      dtype='object')>

In [86]:
this_year.columns.tolist()

['Season',
 'Low_TeamID',
 'High_TeamID',
 'Seed_Dif',
 'COL_diff',
 'DOL_diff',
 'MOR_diff',
 'POM_diff',
 'RPI_diff',
 'RTH_diff',
 'WLK_diff',
 'WOL_diff',
 'AP_ind_x',
 'AP_ind_y',
 'USA_ind_x',
 'USA_ind_y',
 'Avg_PPP_Pre_diff',
 'Avg_PPP_L10_diff',
 'Avg_PPP_Allowed_Pre_diff',
 'Avg_PPP_Allowed_L10_diff',
 'Avg_TS_Perc_Pre_diff',
 'Avg_TS_Perc_L10_diff',
 'Avg_TS_Perc_Allowed_Pre_diff',
 'Avg_TS_Perc_Allowed_L10_diff',
 'Avg_Reb_Marg_Pre_diff',
 'Avg_Reb_Marg_L10_diff',
 'Avg_TO_Perc_Pre_diff',
 'Avg_Forced_TO_Perc_Pre_diff',
 'Avg_Forced_TO_Perc_L10_diff']

In [45]:
y_train = train.iloc[ :,0]
x_train = train.iloc[:,1:]
y_test = test.iloc[ :,0]
x_test = test.iloc[:,1:]

logreg = LogisticRegression()
model = logreg.fit(x_train, y_train)
logit_model=sm.Logit(y_train,x_train)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.517454
         Iterations 7
                                Results: Logit
Model:                   Logit                Pseudo R-squared:     0.253     
Dependent Variable:      Low_team_win         AIC:                  789.8893  
Date:                    2019-03-18 11:07     BIC:                  908.6958  
No. Observations:        713                  Log-Likelihood:       -368.94   
Df Model:                25                   LL-Null:              -494.01   
Df Residuals:            687                  LLR p-value:          5.0894e-39
Converged:               1.0000               Scale:                1.0000    
No. Iterations:          7.0000                                               
------------------------------------------------------------------------------
                              Coef.   Std.Err.    z    P>|z|   [0.025   0.975]
-------------------------------------------------------------

In [46]:
y_pred = logreg.predict(x_test)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test, y_test)))
#Log Loss Calc on Test Set
pred_prob = logreg.predict_proba(x_test)
print("Log Loss",sklearn.metrics.log_loss(y_test,pred_prob))

Accuracy of logistic regression classifier on test set: 0.70
Log Loss 0.5614902425455508


In [47]:
#RFE -Recursive Feature Elimination
model = LogisticRegression()
rfe = RFE(model, 12)
fit = rfe.fit(x_train, y_train)

print("Support:",fit.support_)
print("Ranking:",fit.ranking_)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(fit.score(x_test, y_test)))
pred_prob = fit.predict_proba(x_test)
print("Log Loss",sklearn.metrics.log_loss(y_test,pred_prob))

Support: [ True False False False False False False False False  True  True  True
  True  True  True  True False  True False  True  True False False False
  True False]
Ranking: [ 1  7 12 10 15  8 14 13  6  1  1  1  1  1  1  1  3  1  5  1  1  4 11  2
  1  9]
Accuracy of logistic regression classifier on test set: 0.70
Log Loss 0.5763645676945598


# Modeling Round 3

In [48]:
train3 = train[['Low_team_win','Seed_Dif','MOR_diff','RPI_diff','WOL_diff','Avg_PPP_Pre_diff',
               'Avg_PPP_Allowed_Pre_diff','Avg_TS_Perc_Pre_diff','Avg_TS_Perc_Allowed_Pre_diff','Avg_Reb_Marg_Pre_diff']]
test3 = test[['Low_team_win','Seed_Dif','MOR_diff','RPI_diff','WOL_diff','Avg_PPP_Pre_diff',
               'Avg_PPP_Allowed_Pre_diff','Avg_TS_Perc_Pre_diff','Avg_TS_Perc_Allowed_Pre_diff','Avg_Reb_Marg_Pre_diff']]
y_train3 = train3.iloc[ :,0]
x_train3 = train3.iloc[:,1:]
y_test3 = test3.iloc[ :,0]
x_test3 = test3.iloc[:,1:]

In [49]:
#Logistic Regression Model
logreg = LogisticRegression()
model = logreg.fit(x_train3, y_train3)
logit_model=sm.Logit(y_train3,x_train3)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.527104
         Iterations 7
                               Results: Logit
Model:                  Logit                Pseudo R-squared:     0.239     
Dependent Variable:     Low_team_win         AIC:                  769.6505  
Date:                   2019-03-18 11:08     BIC:                  810.7758  
No. Observations:       713                  Log-Likelihood:       -375.83   
Df Model:               8                    LL-Null:              -494.01   
Df Residuals:           704                  LLR p-value:          1.3276e-46
Converged:              1.0000               Scale:                1.0000    
No. Iterations:         7.0000                                               
-----------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z|   [0.025   0.975]
------------------------------------------------------------------------

In [50]:
y_pred = logreg.predict(x_test3)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test3, y_test3)))
#Log Loss Calc on Test Set
pred_prob = logreg.predict_proba(x_test3)
print("Log Loss",sklearn.metrics.log_loss(y_test3,pred_prob))

Accuracy of logistic regression classifier on test set: 0.72
Log Loss 0.559877652996952


In [51]:
#RFE -Recursive Feature Elimination
model = LogisticRegression()
rfe = RFE(model, 8)
fit = rfe.fit(x_train3, y_train3)

print("Support:",fit.support_)
print("Ranking:",fit.ranking_)

print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(fit.score(x_test3, y_test3)))
pred_prob = fit.predict_proba(x_test3)
print("Log Loss",sklearn.metrics.log_loss(y_test3,pred_prob))

Support: [ True  True False  True  True  True  True  True  True]
Ranking: [1 1 2 1 1 1 1 1 1]
Accuracy of logistic regression classifier on test set: 0.71
Log Loss 0.5607033591582017


In [52]:
train3 = train[['Low_team_win','Seed_Dif','WOL_diff','Avg_PPP_L10_diff',
               'Avg_PPP_Allowed_L10_diff','Avg_TS_Perc_L10_diff','Avg_TS_Perc_Allowed_L10_diff']]
test3 = test[['Low_team_win','Seed_Dif','WOL_diff','Avg_PPP_L10_diff',
               'Avg_PPP_Allowed_L10_diff','Avg_TS_Perc_L10_diff','Avg_TS_Perc_Allowed_L10_diff']]
y_train3 = train3.iloc[ :,0]
x_train3 = train3.iloc[:,1:]
y_test3 = test3.iloc[ :,0]
x_test3 = test3.iloc[:,1:]

#Logistic Regression Model
logreg = LogisticRegression()
model = logreg.fit(x_train3, y_train3)
logit_model=sm.Logit(y_train3,x_train3)
result=logit_model.fit()
print(result.summary2())

Optimization terminated successfully.
         Current function value: 0.535368
         Iterations 7
                               Results: Logit
Model:                  Logit                Pseudo R-squared:     0.227     
Dependent Variable:     Low_team_win         AIC:                  775.4351  
Date:                   2019-03-18 11:08     BIC:                  802.8520  
No. Observations:       713                  Log-Likelihood:       -381.72   
Df Model:               5                    LL-Null:              -494.01   
Df Residuals:           707                  LLR p-value:          1.5458e-46
Converged:              1.0000               Scale:                1.0000    
No. Iterations:         7.0000                                               
-----------------------------------------------------------------------------
                              Coef.  Std.Err.    z    P>|z|   [0.025   0.975]
------------------------------------------------------------------------

In [53]:
y_pred = logreg.predict(x_test3)
print('Accuracy of logistic regression classifier on test set: {:.2f}'.format(logreg.score(x_test3, y_test3)))
#Log Loss Calc on Test Set
pred_prob = logreg.predict_proba(x_test3)
print("Log Loss",sklearn.metrics.log_loss(y_test3,pred_prob))

Accuracy of logistic regression classifier on test set: 0.71
Log Loss 0.5740481025021532


In [54]:
d13.head()

,Season,Low_TeamID,High_TeamID,Low_team_win,Seed_Dif,COL_diff,DOL_diff,MOR_diff,POM_diff,RPI_diff,...,Avg_PPP_Allowed_L10_diff,Avg_TS_Perc_Pre_diff,Avg_TS_Perc_L10_diff,Avg_TS_Perc_Allowed_Pre_diff,Avg_TS_Perc_Allowed_L10_diff,Avg_Reb_Marg_Pre_diff,Avg_Reb_Marg_L10_diff,Avg_TO_Perc_Pre_diff,Avg_Forced_TO_Perc_Pre_diff,Avg_Forced_TO_Perc_L10_diff
0,2003,1411,1421,0,0,-1.0,-19.0,16.0,-5.0,31.0,...,-0.079583,-0.003585,-0.049993,-0.031205,0.013326,4.597291,7.0,-1.269225,2.338849,1.765797
1,2003,1112,1436,1,-15,-142.0,-154.0,-183.0,-141.0,-142.0,...,-0.004224,0.030790,0.043019,-0.028937,-0.027638,-0.513228,-5.3,-1.758093,1.554615,0.142975
2,2003,1113,1272,1,3,19.0,27.0,-1.0,12.0,6.0,...,0.136335,0.022790,0.016552,0.039895,0.097333,0.785714,0.9,0.438123,0.824697,1.527463
3,2003,1141,1166,1,5,17.0,19.0,17.0,23.0,21.0,...,0.043207,0.022643,0.095679,0.030167,0.017834,1.129464,4.7,5.416020,-1.840618,-0.317027
4,2003,1143,1301,1,-1,-28.0,-24.0,12.0,-8.0,-20.0,...,-0.043713,-0.023796,-0.032261,-0.013274,-0.003079,1.740148,0.3,-1.502949,-0.495638,3.062255


In [55]:
#Export dataset to csv
#d13.to_csv('final_df.csv',index=False)

## Logistic Regression Model using Cross Validation

In [56]:
y_train = train.iloc[ :,0]
x_train = train.iloc[:,1:]
y_test = test.iloc[ :,0]
x_test = test.iloc[:,1:]

logreg = LogisticRegression()
params = {'C': np.logspace(start=-5, stop=5, num=10)}
clf3 = GridSearchCV(logreg, params, scoring='neg_log_loss', refit=True)
clf3.fit(x_train, y_train)
print("Best Log Loss Score:",clf3.score(x_train, y_train))

Best Log Loss Score: -0.5366021134313416


In [57]:
clf3.best_estimator_.coef_

array([[-8.63071430e-03,  4.85728451e-03, -6.63107684e-04,
        -7.95038360e-03, -2.26714787e-03, -7.48656977e-03,
         3.97015316e-03, -6.73833182e-03, -1.14560239e-02,
        -4.36183634e-04,  4.25632270e-06, -6.01442570e-05,
         1.10935047e-04,  1.12271928e-04,  1.28590182e-05,
        -1.23079707e-04, -4.52753523e-05, -6.13059499e-06,
        -5.43400991e-05, -1.47287343e-05,  2.86846688e-05,
         3.64764205e-03,  4.03115494e-03, -4.35360957e-03,
         6.93285783e-03,  5.79653387e-03]])

In [58]:
#Log Loss Calc on Test Set
pred_prob = clf3.predict_proba(x_test)
print("Log Loss",sklearn.metrics.log_loss(y_test,pred_prob))

Log Loss 0.5558630698107258


In [59]:
#Accuracy Results
y_pred = clf3.predict(x_test)
print('Confusion Matrix: ')
print(confusion_matrix(y_test, y_pred), '\n')
print(classification_report(y_test, y_pred))

Confusion Matrix: 
[[126  47]
 [ 47 115]] 

             precision    recall  f1-score   support

          0       0.73      0.73      0.73       173
          1       0.71      0.71      0.71       162

avg / total       0.72      0.72      0.72       335



## Submission File

In [ ]:
#sample = pd.read_csv(r'C:\Users\Spelk\Desktop\Georgia Tech Analytics\Stephen Data Science\Data Portfolio\Machine Learning\NCAA Tourney Kaggle Competition\mens-machine-learning-competition-2019\SampleSubmissionStage1.csv')

In [71]:
#Get file path for all datafiles
path = r'C:\Users\Spelk\Desktop\Georgia Tech Analytics\Stephen Data Science\Data Portfolio\Machine Learning\NCAA Tourney Kaggle Competition\Final Submission Files'
os.chdir(path)

#Download all files in wd
data = []

for counter,file in enumerate(os.listdir(path)):
    data.append(pd.read_csv(file,encoding = "ISO-8859-1"))
print('All data loaded!')

#Creating All dfs
Cities = data[0]
Conferences = data[1]
ConferenceTourneyGames = data[2]
GameCities = data[3]
NCAATourneyCompactResults = data[4]
NCAATourneyDetailedResults = data[5]
NCAATourneySeedRoundSlots = data[6]
NCAATourneySeeds = data[7]
NCAATourneySlots = data[8]
RegularSeasonCompactResults = data[9]
RegularSeasonDetailedResults = data[10]
Seasons = data[11]
SecondaryTourneyCompactResults = data[12]
SecondaryTourneyTeams = data[13]
TeamCoaches = data[14]
TeamConferences = data[15]
Teams = data[16]
Massey = data[17]
JP_df = data[18]
sample = data[19]

#Seed to int function
def seed_to_int(seed):
    #get the seed digits
    s_int = int(seed[1:3])
    return s_int

#Extract Ranking difference function
def rank_difs(df,stats_df,system):
    #LowID team's rank
    df2 = df.merge(stats_df[['Season','TeamID',system]],left_on=['Season','Low_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
    #HighID team's rank
    df3 = df2.merge(stats_df[['Season','TeamID',system]],left_on=['Season','High_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
    #Drop multiple team_IDs
    df4 = df3.drop(labels = ['TeamID_x','TeamID_y'],axis = 1)
    
    #Calculate rank difference
    sys1 = system+'_x'
    sys2 = system+'_y'
    diff = system+'_diff'
    df4[diff] = df4[sys1]-df4[sys2]
    df4.drop(labels = [sys1,sys2],axis = 1, inplace=True)
    
    return df4

#Change sample submission file for joins
def sample_setup(sample):
    sample['Season'] = sample.apply(lambda row: row['ID'][0:4], axis=1)
    sample['Low_TeamID'] = sample.apply(lambda row: row['ID'][5:9], axis=1)
    sample['High_TeamID'] = sample.apply(lambda row: row['ID'][10:14], axis=1)
    sample['Season'] = sample['Season'].apply(pd.to_numeric)
    sample['Low_TeamID'] = sample['Low_TeamID'].apply(pd.to_numeric)
    sample['High_TeamID'] = sample['High_TeamID'].apply(pd.to_numeric)
    sample = sample[['Season','Low_TeamID','High_TeamID']]
    return sample



def NCAA_Tourney_Model(Teams,Massey,sample2,NCAATourneySeeds,JP_df):
    rankings = Massey.merge(Teams[['TeamID','TeamName']], how = 'left')
    #Table that will be used to join the end of season rank for each team - by season and system
    EOS_ranking_date = rankings.groupby(['TeamID','Season','SystemName'],
                                        as_index=False)['RankingDayNum'].max()
    #Rankings for each team by season and system - these represent the ranking at the end of the regular season
    EOS_ranking_by_system = EOS_ranking_date.merge(rankings,
                                                   on = ['Season','TeamID','SystemName','RankingDayNum'],
                                                   how = 'left')
    #Drop Columns 'teamName'
    EOS_ranking_by_system.drop(['TeamName','RankingDayNum'],axis=1,inplace=True)
    #Cast rankings
    rankings_pivot = EOS_ranking_by_system.pivot_table(index = ['Season','TeamID'],
                                                       columns = 'SystemName',values = 'OrdinalRank')
    rankings_pivot.reset_index(drop = False,inplace=True)
    relevant_rankings = rankings_pivot[['Season','TeamID','COL','DOL','MOR','POM','RPI','RTH','WLK','WOL','AP','USA']]
    #Creating binary variables for AP and USA - drop original variables inplace
    relevant_rankings['AP_ind'] = np.where(relevant_rankings['AP'] > 0,1,0)
    relevant_rankings['USA_ind'] = np.where(relevant_rankings['USA'] > 0,1,0)
    relevant_rankings = relevant_rankings.drop(['AP','USA'],axis=1)
    #Fill NAs with the mean of each column (if they're NA, they're most likely not in the tournament anyways)
    relevant_rankings.fillna(relevant_rankings.mean(), inplace=True)
    
    
    #This section depends on if you want the sample submission file or model building dataset
    NCAA_results = sample2 
    
    NCAATourneySeeds['seed_int'] = NCAATourneySeeds.Seed.apply(seed_to_int)
    NCAATourneySeeds.drop(labels=['Seed'], inplace=True, axis=1)
    NCAA_df = NCAA_results.merge(NCAATourneySeeds[['Season','TeamID','seed_int']], left_on=['Season','Low_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
    NCAA_df2 = NCAA_df.merge(NCAATourneySeeds[['Season','TeamID','seed_int']], left_on=['Season','High_TeamID'],
                   right_on=['Season','TeamID'], how = 'left')
    NCAA_df3 = NCAA_df2.drop(labels = ['TeamID_x','TeamID_y'],axis = 1)
    NCAA_df3.rename(columns = {'seed_int_x':'Low_Seed','seed_int_y':'High_Seed'},inplace=True)
    NCAA_df3['Seed_Dif'] = NCAA_df3['Low_Seed']-NCAA_df3['High_Seed']
    NCAA_df3.drop(labels = ['Low_Seed','High_Seed'],axis = 1, inplace=True)
    #Can only use tourney results from 2003-2018 --> don't have rankings back farther than that
    mask_2003 = NCAA_df3['Season']>2002
    NCAA_df4 = NCAA_df3[mask_2003]
    d1 = rank_difs(NCAA_df4,relevant_rankings,'COL')
    d2 = rank_difs(d1,relevant_rankings,'DOL')
    d3 = rank_difs(d2,relevant_rankings,'MOR')
    d4 = rank_difs(d3,relevant_rankings,'POM')
    d5 = rank_difs(d4,relevant_rankings,'RPI')
    d6 = rank_difs(d5,relevant_rankings,'RTH')
    d7 = rank_difs(d6,relevant_rankings,'WLK')
    d8 = rank_difs(d7,relevant_rankings,'WOL')
    #AP for both teams
    d9 = d8.merge(relevant_rankings[['Season','TeamID','AP_ind']],left_on= ['Season','Low_TeamID'],
         right_on = ['Season','TeamID'], how = 'left')
    d10 = d9.merge(relevant_rankings[['Season','TeamID','AP_ind']],left_on= ['Season','High_TeamID'],
         right_on = ['Season','TeamID'], how = 'left')

    #USA for both teams
    df11 = d10.merge(relevant_rankings[['Season','TeamID','USA_ind']],left_on= ['Season','Low_TeamID'],
         right_on = ['Season','TeamID'], how = 'left')
    final_df = df11.merge(relevant_rankings[['Season','TeamID','USA_ind']],left_on= ['Season','High_TeamID'],
         right_on = ['Season','TeamID'], how = 'left')
    final_df = final_df.drop(labels = ['TeamID_x','TeamID_y'],axis = 1)
    
    #Part 2 - JPs data
    #Stats at the end of the regular season for every single team - includes last 10 stats
    TeamID_max_game = JP_df.groupby(['Season','TeamID'],as_index=False)['Game'].max()
    JP2 = JP_df[['Season', 'TeamID', 'OppID', 'Game','Avg_PPP_Pre','Avg_PPP_L10','Avg_PPP_Allowed_Pre', 'Avg_PPP_Allowed_L10',
             'Avg_TS_Perc_Pre', 'Avg_TS_Perc_L10', 'Avg_TS_Perc_Allowed_Pre', 'Avg_TS_Perc_Allowed_L10',
             'Avg_Reb_Marg_Pre', 'Avg_Reb_Marg_L10', 'Avg_TO_Perc_Pre', 'Avg_Forced_TO_Perc_L10', 'Avg_Forced_TO_Perc_Pre']]
    Max_game_stats = TeamID_max_game.merge(JP2,left_on=['Season','TeamID','Game'],right_on=['Season','TeamID','Game'], how = 'left')
    
    d1 = rank_difs(final_df,Max_game_stats,'Avg_PPP_Pre')
    d2 = rank_difs(d1,Max_game_stats,'Avg_PPP_L10')
    d3 = rank_difs(d2,Max_game_stats,'Avg_PPP_Allowed_Pre')
    d4 = rank_difs(d3,Max_game_stats,'Avg_PPP_Allowed_L10')
    d5 = rank_difs(d4,Max_game_stats,'Avg_TS_Perc_Pre')
    d6 = rank_difs(d5,Max_game_stats,'Avg_TS_Perc_L10')
    d7 = rank_difs(d6,Max_game_stats,'Avg_TS_Perc_Allowed_Pre')
    d8 = rank_difs(d7,Max_game_stats,'Avg_TS_Perc_Allowed_L10')
    d9 = rank_difs(d8,Max_game_stats,'Avg_Reb_Marg_Pre')
    d10 = rank_difs(d9,Max_game_stats,'Avg_Reb_Marg_L10')
    d11 = rank_difs(d10,Max_game_stats,'Avg_TO_Perc_Pre')
    d12 = rank_difs(d11,Max_game_stats,'Avg_Forced_TO_Perc_Pre')
    d13 = rank_difs(d12,Max_game_stats,'Avg_Forced_TO_Perc_L10')
    return d13

All data loaded!


In [72]:
sample.head()

,ID,Pred
0,2019_1101_1113,0.5
1,2019_1101_1120,0.5
2,2019_1101_1124,0.5
3,2019_1101_1125,0.5
4,2019_1101_1133,0.5


In [73]:
sample2 = sample_setup(sample)
this_year = NCAA_Tourney_Model(Teams,Massey,sample2,NCAATourneySeeds,JP_df)

C:\Users\Spelk\Anaconda3\lib\site-packages\ipykernel_launcher.py:90: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [75]:
this_year.head()

,Season,Low_TeamID,High_TeamID,Seed_Dif,COL_diff,DOL_diff,MOR_diff,POM_diff,RPI_diff,RTH_diff,...,Avg_PPP_Allowed_L10_diff,Avg_TS_Perc_Pre_diff,Avg_TS_Perc_L10_diff,Avg_TS_Perc_Allowed_Pre_diff,Avg_TS_Perc_Allowed_L10_diff,Avg_Reb_Marg_Pre_diff,Avg_Reb_Marg_L10_diff,Avg_TO_Perc_Pre_diff,Avg_Forced_TO_Perc_Pre_diff,Avg_Forced_TO_Perc_L10_diff
0,2019,1101,1113,4,81.0,77.0,115.0,89.0,0.0,103.0,...,-0.083675,0.010950,-0.000670,0.012272,-0.030399,-5.685484,-0.2,-0.765430,3.517256,1.648600
1,2019,1101,1120,10,97.0,105.0,161.0,137.0,0.0,137.0,...,-0.126021,-0.014119,-0.015929,-0.013550,-0.063079,0.765152,8.1,0.061507,-1.723788,-1.504607
2,2019,1101,1124,6,77.0,74.0,129.0,116.0,0.0,116.0,...,-0.174973,0.018925,0.030290,0.022757,-0.034541,-7.104839,-5.5,-1.434055,4.141925,7.376416
3,2019,1101,1125,4,74.0,80.0,122.0,98.0,0.0,85.0,...,-0.045697,-0.051888,-0.099455,0.022742,0.004239,-4.508621,-7.1,1.248062,5.836275,5.166658
4,2019,1101,1133,0,-32.0,-26.0,1.0,-16.0,0.0,2.0,...,-0.039414,0.024417,-0.008809,0.018523,0.012132,-1.562500,-3.9,-1.586983,4.163957,6.383559


In [76]:
final_df_preds = this_year.drop(['Season','Low_TeamID','High_TeamID'], axis =1)

In [92]:
pred_prob = clf3.predict_proba(final_df_preds)

In [105]:
column_entry = []
for i in pred_prob:
    column_entry.append(i[1])

In [107]:
this_year['preds'] = column_entry

In [112]:
sample3 = sample2.merge(this_year,on=['Season','Low_TeamID','High_TeamID'], how = 'left')

In [120]:
submission = sample3[['Season','Low_TeamID','High_TeamID', 'preds']]
submission['ID'] = submission['Season'].astype(str)+'_'+submission['Low_TeamID'].astype(str)+'_'+submission['High_TeamID'].astype(str)

C:\Users\Spelk\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [124]:
final = submission[['ID','preds']]
final.to_csv('submission_file.csv', index= False)
this_year.to_csv('final_df.csv', index = False)

In [133]:
df = submission.merge(Teams, left_on = 'Low_TeamID', right_on= 'TeamID', how = 'left')
df2 = df.merge(Teams, left_on = 'High_TeamID',right_on = 'TeamID',how = 'left')
df3 = df2[['TeamName_x','TeamName_y','preds']]

In [134]:
df3.to_csv('game_predictions.csv', index = False)

In [ ]:
import bracketeer
import os
path = r'C:\Users\Spelk\Desktop\Georgia Tech Analytics\Stephen Data Science\Data Portfolio\Machine Learning\NCAA Tourney Kaggle Competition\Final Submission Files'
os.chdir(path)

from bracketeer import build_bracket


b = build_bracket(
        outputPath='output.png',
        teamsPath='Teams.csv',
        seedsPath='NCAATourneySeeds.csv',
        submissionPath='submission_file.csv',
        slotsPath='NCAATourneySlots.csv',
        year=2019
)